In [ ]:
# ================================
# Step 1: Setup Environment
# ================================
!pip install -q ultralytics
# Mount Google Drive for data persistence
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: Tesla T4


In [ ]:
# ================================
# Step 2: Upload and Extract Dataset
# ================================
import json
import os
import cv2

# Define paths
data_dir = "/content/PigDataset"
os.makedirs(data_dir, exist_ok=True)
images_dir = os.path.join(data_dir, "images")
labels_dir = os.path.join(data_dir, "labels")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Upload the outputs.json and video file manually or through Drive
outputs_path = os.path.join(data_dir, "/content/drive/MyDrive/EdinburghPig/000002/output.json")
video_path = os.path.join(data_dir, "/content/drive/MyDrive/EdinburghPig/000002/color.mp4")

In [ ]:
# ! rm -rf PigBehaviorDataset/

In [ ]:
# ! rm -rf PigDataset/images/*
# ! rm -rf PigDataset/labels/*

In [ ]:
# ================================
# Step 3: Extract Frames from Video
# ================================
def extract_frames(video_path, output_folder, frame_step=3):
    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save every third frame starting from the first
        if frame_count % frame_step == 0:
            frame_name = f"frame_{count:04d}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_name), frame)
            count += 1

        frame_count += 1

    cap.release()
    print(f"Extracted {count} frames from video.")

# Use the modified function
extract_frames(video_path, images_dir, frame_step=3)

Extracted 600 frames from video.


In [ ]:
import os
import json
import cv2

def create_behavior_dataset(outputs_path, images_dir, dataset_dir):
    # Create directories for the behavior dataset
    standing_dir = os.path.join(dataset_dir, "standing")
    lying_dir = os.path.join(dataset_dir, "lying")
    os.makedirs(standing_dir, exist_ok=True)
    os.makedirs(lying_dir, exist_ok=True)

    # Load the annotations (output.json)
    with open(outputs_path, 'r') as f:
        data = json.load(f)

    total_pigs = len(data['objects'])
    print(f"Total pigs found: {total_pigs}")

    image_count = 0

    # Process each pig separately
    for obj in data['objects']:
        pig_id = int(obj['id'])
        print(f"Processing Pig ID: {pig_id}")

        for frame in obj['frames']:
            frame_number = frame['frameNumber']
            bbox = frame['bbox']
            x, y, w, h = bbox['x'], bbox['y'], bbox['width'], bbox['height']
            behavior = frame['behaviour']  # Get the behavior label

            # Only proceed if the behavior is "standing" or "lying"
            if behavior not in ["standing", "lying"]:
                continue

            # Path of the frame image
            img_path = os.path.join(images_dir, f"frame_{frame_number:04d}.jpg")
            if not os.path.exists(img_path):
                print(f"Frame {frame_number} does not exist. Skipping.")
                continue

            # Read image and crop the region
            img = cv2.imread(img_path)
            if img is None:
                print(f"Frame {frame_number} is missing. Skipping.")
                continue

            # Calculate crop coordinates
            x1 = int(x)
            y1 = int(y)
            x2 = int(x + w)
            y2 = int(y + h)

            # Ensure the crop coordinates are within image boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(img.shape[1], x2)
            y2 = min(img.shape[0], y2)

            cropped_img = img[y1:y2, x1:x2]

            # Save the cropped image in the corresponding directory
            if behavior == "standing":
                save_path = os.path.join(standing_dir, f"pig_{pig_id}_frame_{frame_number:04d}.jpg")
            elif behavior == "lying":
                save_path = os.path.join(lying_dir, f"pig_{pig_id}_frame_{frame_number:04d}.jpg")

            cv2.imwrite(save_path, cropped_img)
            image_count += 1

    print("\n✅ Behavior Dataset Creation Completed")
    print(f"Total Images Saved: {image_count}")
    print(f"Standing Images: {len(os.listdir(standing_dir))}")
    print(f"Lying Images: {len(os.listdir(lying_dir))}")


# Usage Example
outputs_path = "/content/drive/MyDrive/EdinburghPig/000002/output.json"  # Path to your JSON file
images_dir = "/content/PigDataset/images"         # Directory of your frame images
dataset_dir = "/content/PigBehaviorDataset"       # Directory to save the cropped behavior images

create_behavior_dataset(outputs_path, images_dir, dataset_dir)


Total pigs found: 8
Processing Pig ID: 3
Processing Pig ID: 4
Processing Pig ID: 6
Processing Pig ID: 7
Processing Pig ID: 2
Processing Pig ID: 1
Processing Pig ID: 5
Processing Pig ID: 0

✅ Behavior Dataset Creation Completed
Total Images Saved: 609
Standing Images: 551
Lying Images: 58


In [ ]:
import os
import json
import cv2

# ================================
# Enhanced Function: Parse Annotations (Ensure 600 Frames)
# ================================
def parse_annotations(outputs_path, images_dir, labels_dir):
    with open(outputs_path, 'r') as f:
        data = json.load(f)

    total_pigs = len(data['objects'])
    print(f"Total pigs found: {total_pigs}")

    frame_count = 0
    label_count = 0

    # Process each pig separately
    for obj in data['objects']:
        pig_id = int(obj['id'])
        print(f"Processing Pig ID: {pig_id}")

        # Initialize last known bounding box
        last_bbox = None
        frames_dict = {frame['frameNumber']: frame['bbox'] for frame in obj['frames']}

        # Ensure 600 frames with interpolation
        for frame_number in range(600):
            if frame_number in frames_dict:
                last_bbox = frames_dict[frame_number]
            elif last_bbox is not None:
                frames_dict[frame_number] = last_bbox

            if last_bbox is not None:
                bbox = last_bbox
                x, y, w, h = bbox['x'], bbox['y'], bbox['width'], bbox['height']

                # Path of the frame image
                img_path = os.path.join(images_dir, f"frame_{frame_number:04d}.jpg")
                if not os.path.exists(img_path):
                    print(f"Frame {frame_number} does not exist. Skipping.")
                    continue

                # Read image to get dimensions
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Frame {frame_number} is missing. Skipping.")
                    continue

                img_h, img_w, _ = img.shape

                # Normalize bbox coordinates (YOLO format)
                x_center = (x + w / 2) / img_w
                y_center = (y + h / 2) / img_h
                norm_w = w / img_w
                norm_h = h / img_h

                # Save label file (using 'a' to allow multiple pigs in one frame)
                label_path = os.path.join(labels_dir, f"frame_{frame_number:04d}.txt")
                with open(label_path, 'a') as label_file:
                    label_file.write(f"{pig_id} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")
                    label_count += 1

            frame_count += 1

    print("\n==============================")
    print(f"Annotation parsing completed.")
    print(f"Total Frames Processed: {frame_count} (Expected: 600)")
    print(f"Total Labels Created: {label_count}")
    print(f"Annotations saved in {labels_dir}.")


# Run the optimized function with debugging
parse_annotations(outputs_path, images_dir, labels_dir)

Total pigs found: 8
Processing Pig ID: 3
Processing Pig ID: 4
Processing Pig ID: 6
Processing Pig ID: 7
Processing Pig ID: 2
Processing Pig ID: 1
Processing Pig ID: 5
Processing Pig ID: 0

Annotation parsing completed.
Total Frames Processed: 4800 (Expected: 600)
Total Labels Created: 4800
Annotations saved in /content/PigDataset/labels.


In [ ]:
! ls -1 /content/PigDataset/images | wc -l

600


In [ ]:
! ls -1 /content/PigDataset/labels/ | wc -l

600


In [ ]:
# ! rm -rf /content/PigDataset/images/*

In [ ]:
# ! rm -rf /content/PigDataset/labels/*

In [ ]:
import os

# Define paths
data_dir = "/content/PigDataset"

# Create a YAML configuration for YOLO training
dataset_yaml = f"""path: {data_dir}
train: images  # Use all images in 'images' directory for training
val: images    # Use the same for validation (you can split later)
test: images   # Use the same for testing (optional)
nc: 8           # Number of classes (8 pigs, IDs 0-7)
names:
  0: pig_0
  1: pig_1
  2: pig_2
  3: pig_3
  4: pig_4
  5: pig_5
  6: pig_6
  7: pig_7
"""

# Save this YAML file in the data directory
yaml_path = os.path.join(data_dir, "pig_dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

print(f"Dataset YAML file created at: {yaml_path}")


Dataset YAML file created at: /content/PigDataset/pig_dataset.yaml


In [ ]:
from ultralytics import YOLO
# Load the YOLOv9 model (YOLOv8 latest but effectively YOLOv9)
model = YOLO("yolov9c.pt")  # Using the nano model (fast) - change to 'yolov8s.pt' for better accuracy

# Start training
model.train(
    data=yaml_path,        # Path to the custom dataset YAML file
    epochs=1,             # Number of training epochs
    imgsz=640,             # Image size (adjust for better accuracy)
    batch=4,              # Batch size (adjust based on GPU memory)
    project="/content/YoloRuns",  # Save runs to Google Drive
    name="pig_yolo_v9_test_1",    # Custom name for the run
    workers=4,             # Number of workers (adjust for speed)
    device=0                # Use GPU (CUDA)
)


Ultralytics 8.3.138 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/PigDataset/pig_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9c.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pig_yolo_v9_test_12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose

100%|██████████| 5.35M/5.35M [00:00<00:00, 65.6MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3064.7±600.2 MB/s, size: 325.9 KB)


train: Scanning /content/PigDataset/labels.cache... 600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

        1/1         3G      1.203       1.73      1.322         53        640: 100%|██████████| 150/150 [00:41<00:00,  3.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [00:12<00:00,  5.84it/s]


                   all        600       4800      0.812      0.774      0.864      0.597

1 epochs completed in 0.017 hours.
Optimizer stripped from /content/YoloRuns/pig_yolo_v9_test_12/weights/last.pt, 51.6MB
Optimizer stripped from /content/YoloRuns/pig_yolo_v9_test_12/weights/best.pt, 51.6MB

Validating /content/YoloRuns/pig_yolo_v9_test_12/weights/best.pt...
Ultralytics 8.3.138 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9c summary (fused): 156 layers, 25,325,416 parameters, 0 gradients, 102.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 75/75 [00:09<00:00,  8.08it/s]


                   all        600       4800      0.811      0.775      0.864      0.597
                 pig_0        600        600      0.989      0.911      0.981       0.63
                 pig_1        600        600      0.567      0.762      0.725       0.49
                 pig_2        600        600       0.86      0.801      0.898      0.645
                 pig_3        600        600      0.905      0.608      0.832      0.565
                 pig_4        600        600      0.852      0.883      0.928      0.685
                 pig_5        600        600      0.851      0.642      0.822      0.588
                 pig_6        600        600      0.792       0.62      0.775       0.49
                 pig_7        600        600      0.673      0.972      0.949      0.683
Speed: 0.2ms preprocess, 7.8ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /content/YoloRuns/pig_yolo_v9_test_12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78d3363bb5d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [ ]:
# Display the training results
from IPython.display import Image, display

# Display the training curve
results_image = "/content/YoloRuns/pig_yolo_v9_test_1/results.png"
display(Image(results_image))


FileNotFoundError: No such file or directory: '/content/YoloRuns/pig_yolo_v9_test_1/results.png'

FileNotFoundError: No such file or directory: '/content/YoloRuns/pig_yolo_v9_test_1/results.png'

<IPython.core.display.Image object>

In [ ]:
from ultralytics import YOLO

# Load the trained model (best weights)
trained_model = YOLO("/content/YoloRuns/pig_yolo_v9_test_12/weights/best.pt")

# Evaluate the model on validation set
metrics = trained_model.val()  # This will automatically use the validation set specified in your YAML file

# # Display evaluation metrics
# print(f"Precision: {metrics['precision']:.4f}")
# print(f"Recall: {metrics['recall']:.4f}")
# print(f"mAP@0.5: {metrics['map50']:.4f}")
# print(f"mAP@0.5:0.95: {metrics['map']:.4f}")


Ultralytics 8.3.138 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9c summary (fused): 156 layers, 25,325,416 parameters, 0 gradients, 102.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3619.1±837.0 MB/s, size: 327.9 KB)


val: Scanning /content/PigDataset/labels.cache... 600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:15<00:00,  2.42it/s]


                   all        600       4800      0.813      0.774      0.864      0.597
                 pig_0        600        600      0.989       0.91      0.981      0.631
                 pig_1        600        600      0.567      0.761      0.725      0.489
                 pig_2        600        600       0.86      0.801      0.899      0.646
                 pig_3        600        600      0.908      0.607      0.832      0.566
                 pig_4        600        600      0.856      0.884      0.928      0.685
                 pig_5        600        600      0.857      0.642      0.822      0.588
                 pig_6        600        600      0.793      0.618      0.774      0.489
                 pig_7        600        600      0.675      0.972       0.95      0.684
Speed: 0.5ms preprocess, 18.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val


In [ ]:
# Display key metrics
# Access metrics using the methods provided by the metrics object
print(f"Precision: {metrics.box.mp:.4f}") # Using metrics.box.mp for mean precision
print(f"Recall: {metrics.box.mr:.4f}")   # Using metrics.box.mr for mean recall
print(f"mAP@0.5: {metrics.box.map50:.4f}") # Using metrics.box.map50 for mAP@0.5
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}") # Using metrics.box.map for mAP@0.5:0.95

Precision: 0.8131
Recall: 0.7742
mAP@0.5: 0.8638
mAP@0.5:0.95: 0.5973


In [ ]:
# Use the trained model for inference on a test video
test_video = "/content/drive/MyDrive/EdinburghPig/000009/color.mp4"  # Your test video
results = trained_model.track(
    source=test_video,     # Path to the test video
    conf=0.25,             # Confidence threshold
    iou=0.45,              # IOU threshold
    show=True,             # Display video
    save=True,             # Save results
    save_txt=True,         # Save detection results as text files
    save_conf=True,        # Save confidence scores
    save_crop=False,       # Save cropped detections
    tracker="bytetrack.yaml"  # Use ByteTrack for multi-object tracking
)

# Display the saved result video
# from IPython.display import Video
# result_video = "/content/drive/MyDrive/YoloRuns/pig_yolo_v9_test_1/track.mp4"
# Video(result_video, embed=True)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.7s, installed 1 package: ['lap>=0.5.12']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1800) /content/drive/MyDrive/Ed

KeyboardInterrupt: 

In [ ]:
!cp /content/runs/detect/track/color.avi /content/drive/MyDrive/EdinburghPig

In [ ]:
!pip install motmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
import motmetrics as mm
import os

# Initialize the metric accumulator
acc = mm.MOTAccumulator(auto_id=True)

# Path to the tracking results (YOLO tracking results are stored in txt files)
tracking_results_dir = "/content/runs/detect/track/labels"  # Change to your tracking folder

# Function to parse tracking results
def parse_tracking_results(tracking_results_dir):
    frame_data = {}

    for file in os.listdir(tracking_results_dir):
        if file.endswith(".txt"):
            frame_number = int(file.split(".")[0].split("_")[-1])
            frame_data[frame_number] = []

            with open(os.path.join(tracking_results_dir, file), "r") as f:
                for line in f.readlines():
                    data = line.strip().split()
                    track_id = int(data[0])
                    frame_data[frame_number].append(track_id)

    return frame_data

# Parse tracking results
tracking_data = parse_tracking_results(tracking_results_dir)

# Check if tracking data is empty
if len(tracking_data) == 0:
    print("No tracking data found. Please check the tracking directory.")
else:
    # Generate ground truth and predictions
    previous_frame_tracks = {}
    fragmentation = 0

    for frame, track_ids in tracking_data.items():
        gt_ids = list(previous_frame_tracks.keys())
        pred_ids = track_ids

        # Calculate distances (zero because we are only tracking identity)
        distances = [[0] * len(pred_ids) for _ in gt_ids] if gt_ids else []

        # Update MOT accumulator
        acc.update(
            gt_ids,                 # Ground Truth IDs
            pred_ids,               # Predicted IDs
            distances               # Zero distances (identity match)
        )

        # Track continuity for fragmentation
        for track_id in pred_ids:
            if track_id not in previous_frame_tracks:
                previous_frame_tracks[track_id] = frame
            else:
                if frame - previous_frame_tracks[track_id] > 1:  # Fragmentation occurs
                    fragmentation += 1
                previous_frame_tracks[track_id] = frame

    # Calculate MOTA and IDF1 using MOTMetrics
    mh = mm.metrics.create()
    summary = mh.compute(
        acc,
        metrics=['mota', 'idf1', 'idp', 'idr', 'num_switches'],
        name='Tracking'
    )

    # Add Fragmentation to the summary
    summary['Fragmentation'] = fragmentation

    # Display the results
    print("\n================ Tracking Metrics ================")
    print(summary)



================ Tracking Metrics ================
              mota      idf1     idp       idr  num_switches  Fragmentation
Tracking  0.525713  1.084728  1.6411  1.185998          3266           4533


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from torchvision import transforms, models
from PIL import Image
import time

# Initialize YOLO model
model = YOLO('/content/YoloRuns/pig_yolo_v9_test_12/weights/best.pt')  # Replace with your custom YOLO model path

# Device setup for GPU acceleration (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Pre-trained EfficientNet-B0 Model (from torchvision)
effnet_model = models.efficientnet_b0(pretrained=True)
effnet_model.classifier[1] = torch.nn.Linear(effnet_model.classifier[1].in_features, 2)  # Binary: Lying or Standing
effnet_model.to(device).eval()

# Image transformation for EfficientNet-B0
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Feeder and Drinker Zones (Define Manually or Interactively)
FEEDER_ZONE = [(368, 473), (374, 243)]
DRINKER_ZONE = [(192, 549), (142, 167)]


# Function to check if a point is in a defined zone
def is_in_zone(centroid, zone):
    (x, y) = centroid
    return zone[0][0] <= x <= zone[1][0] and zone[0][1] <= y <= zone[1][1]

# Function for behavior classification
def classify_behavior(results, frame, centroid_history):
    behaviors = {}

    if results.boxes is None:  # No detections in this frame
        return behaviors

    for box in results.boxes:
        # Extract bounding box, convert tensor to list if necessary
        box_coords = box.xyxy[0].tolist() if isinstance(box.xyxy[0], torch.Tensor) else box.xyxy[0]
        x1, y1, x2, y2 = map(int, box_coords[:4])  # Bounding Box Coordinates
        track_id = int(box.id) if box.id is not None else -1  # Handle missing ID

        # Skip if no valid ID
        if track_id == -1:
            continue

        # Calculate centroid of the bounding box
        centroid = ((x1 + x2) // 2, (y1 + y2) // 2)

        # Update centroid history
        if track_id not in centroid_history:
            centroid_history[track_id] = []
        centroid_history[track_id].append(centroid)

        # Calculate the average bounding box size (for dynamic threshold)
        box_width = abs(x2 - x1)
        box_height = abs(y2 - y1)
        avg_box_size = (box_width + box_height) / 2

        # Set a dynamic movement threshold as 5% of the average box size
        dynamic_threshold = avg_box_size * 0.1  # 5% of the box size

        # Calculate movement for behavior classification
        if len(centroid_history[track_id]) > 5:  # 5 frames (~1 second)
            movement = np.linalg.norm(
                np.array(centroid_history[track_id][-1]) -
                np.array(centroid_history[track_id][-6])
            )

            if movement < dynamic_threshold:  # Dynamic Movement Threshold
                # Check if in feeding or drinking zones
                if is_in_zone(centroid, FEEDER_ZONE):
                    behaviors[track_id] = 'Feeding'
                elif is_in_zone(centroid, DRINKER_ZONE):
                    behaviors[track_id] = 'Drinking'
                else:
                    # Use EfficientNet-B0 for Lying/Standing Classification
                    crop = frame[y1:y2, x1:x2]
                    if crop.size != 0 and crop.shape[0] > 0 and crop.shape[1] > 0:
                        crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
                        input_tensor = transform(crop_pil).unsqueeze(0).to(device)
                        with torch.no_grad():
                            output = effnet_model(input_tensor)
                            _, pred = torch.max(output, 1)
                            pred_class = pred.item()
                            behaviors[track_id] = 'Lying' if pred_class == 0 else 'Standing'
            else:
                behaviors[track_id] = 'Moving'
        else:
            behaviors[track_id] = 'Moving'

    return behaviors

# Video Processing with YOLO + Behavior Classification
cap = cv2.VideoCapture('/content/drive/MyDrive/EdinburghPig/000009/color.mp4')
centroid_history = {}

# Define Video Writer for Saving Output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30, (int(cap.get(3)), int(cap.get(4))))

fps_counter = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results_list = model.track(frame, persist=True, verbose=False)
    if len(results_list) == 0:
        continue
    results = results_list[0]

    # Classify Pig Behaviors
    behaviors = classify_behavior(results, frame, centroid_history)

    # Draw Bounding Boxes and Behavior Labels
    if results.boxes is not None:
        for box in results.boxes:
            box_coords = box.xyxy[0].tolist() if isinstance(box.xyxy[0], torch.Tensor) else box.xyxy[0]
            x1, y1, x2, y2 = map(int, box_coords[:4])
            track_id = int(box.id) if box.id is not None else -1
            behavior = behaviors.get(track_id, "Unknown")

            # Draw bounding box and centroid
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            centroid = ((x1 + x2) // 2, (y1 + y2) // 2)
            cv2.circle(frame, centroid, 5, (0, 0, 255), -1)  # Red dot for centroid
            cv2.putText(frame, f'ID {track_id}: {behavior}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show the Frame
    # cv2.imshow("Pig Behavior Classification", frame)
    out.write(frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Using device: cuda


In [ ]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'pig_0', 1: 'pig_1', 2: 'pig_2', 3: 'pig_3', 4: 'pig_4', 5: 'pig_5', 6: 'pig_6', 7: 'pig_7'}
 obb: None
 orig_img: array([[[ 33,  54,  64],
         [ 30,  51,  61],
         [ 45,  43,  54],
         ...,
         [ 30,  46,  40],
         [ 25,  47,  46],
         [ 26,  48,  47]],
 
        [[ 31,  52,  62],
         [ 30,  51,  61],
         [ 48,  46,  57],
         ...,
         [ 29,  45,  39],
         [ 25,  47,  46],
         [ 25,  47,  46]],
 
        [[ 33,  51,  62],
         [ 34,  52,  63],
         [ 50,  53,  63],
         ...,
         [ 28,  47,  38],
         [ 26,  49,  46],
         [ 26,  49,  46]],
 
        ...,
 
        [[ 11,  79, 109],
         [  9,  77, 107],
         [ 10,  75, 101],
         ...,
         [116, 146, 158],
         [116, 146, 162],
         [118, 148, 164]],
 
        [[  2,  74, 104],
 

In [ ]:

# # ================================
# # Step 7: Display Results
# # ================================
# from IPython.display import Video

# result_video = "/content/YoloRuns/pig_yolo_v9_test_14/color.avi"
# Video(result_video, embed=True)

In [ ]:
!cp /content/output_video.mp4 /content/drive/MyDrive/EdinburghPig